In [26]:
import re
import os
import pickle
import spacy
nlp = spacy.load("en_core_web_lg", disable = 'ner')

##### Збираю речення з корпусу. У якості тренувального корпусу я використала дві книги - Гру Престолів та Портрет Доріана Грея. Додатково відфільтровую речення капслоком (назви розділів), речення з прямою мовою, так як Spacy sentence tokenizer їх дуже погано парсить. Загальна кількість тренувального корупусу - 59282 речень

In [28]:
def get_data (text):
    result = []
    pos_tags = []
    doc = nlp(text)
    for sent in doc.sents:
        text = sent.text
        # process sentence tokenizer's mistakes - poor parsing of direct speech
        # process names of chapters, dates etc
        if not re.search(r'["“]', text) and not re.match(r'[a-z,-]', text) \
        and not re.search(r'\b[A-Z]+\b', text):
            result.append(text)
    return result


def gather_sentences (path):
    sentences = []

    for filename in os.listdir(path):
        with open(os.path.join(path, filename), 'r') as f: 
            paragraphs = f.read().split('\n\n')
            for paragraph in paragraphs:
                sentences.extend(get_data(paragraph.replace('\n', ' ').strip()))
                
    return sentences        

sentences = gather_sentences('train_data/')
len(sentences)

59282

In [10]:
with open('train_data/sentences.pickle', 'wb') as handle:
    pickle.dump(sentences, handle, protocol=pickle.HIGHEST_PROTOCOL)

##### У кожному реченні видаляю кінцеву пунктуацію

In [29]:
for sent in sentences:
    sent = re.sub('[?!(...).]["”]?$', '', sent)
    clean_sentences.append(sent)

##### Приводжу до нижнього регістру перше слово кожного третього речення (якщо воно не починається з "I" )

In [19]:
def lowercase_sents (sentences):
    lowercased_sent = []

    for i, x in enumerate(sentences):
        sent = clean_sentences[i]

        if i % 3 == 0:
            first_letter = sent.split()[0]

            if first_letter != "I": 
                sent = first_letter.lower() + " " + ' '.join(sent.split()[1:])

        lowercased_sent.append(sent)
    return lowercased_sent

lowercased = lowercase_sents(clean_sentences)

##### Формую пари з 2/3/4 склеєних речень

In [32]:
def form_pairs (data, pair_number):
    return [data[k:k+pair_number] for k in range(0, len(data), pair_number)]


def combine_sentences (sentences):
    
    result = []
    # 70% of sentences -> tuples with 2 elements
    len_batch_1 = round(len(sentences) * 0.7)
    result.extend(form_pairs(sentences[:len_batch_1], 2))

    # 20% of sentences -> tuples with 3 elements
    len_batch_2 = round(len(sentences) * 0.2) + len_batch_1
    result.extend(form_pairs(sentences[len_batch_1:len_batch_2],3))

    # 10% of sentences -> tuples with 4 elements
    result.extend(form_pairs(sentences[len_batch_2:],4))

    return result

sentence_combinations = combine_sentences(lowercased)
sentence_combinations[4500:4505]

[['robb and Bran and Rickon were his father’s sons, and he loved them still, yet Jon knew that he had never truly been one of them',
  'Catelyn Stark had seen to that'],
 ['The grey walls of Winterfell might still haunt his dreams, but Castle Black was his life now, and his brothers were Sam and Grenn and Halder and Pyp and the other cast-outs who wore the black of the Night’s Watch',
  'he wondered if he would ever see Benjen Stark again, to tell him'],
 ['This cursed heat had half the city in a fever to start, and now with all these visitors… last night we had a drowning, a tavern riot, three knife fights, a rape, two fires, robberies beyond count, and a drunken horse race down the Street of the Sisters',
  'The night before a woman’s head was found in the Great Sept, floating in the rainbow pool'],
 ['no one seems to know how it got there or who it belongs to',
  'Lord Renly Baratheon was less sympathetic'],
 ['Stout, jowly Janos Slynt puffed himself up like an angry frog, his bald 

##### До вже склеєних речень (фактично - список списків) додаю лейбли True - якщо це кінець речення, False - початок/середина речення

In [33]:
def add_labels (sentences):
    data = []

    for pair in sentences:
        sentence_result = []
        for sent in pair:
            doc = nlp(sent)
            for i, token in enumerate(doc):

                #mark the last token
                if i == len(doc) - 1:
                    sentence_result.append((token.text, "True"))
                else:
                    sentence_result.append((token.text, "False"))
        data.append(sentence_result)

    return data

train_data = add_labels(sentence_combinations)

In [35]:
train_data[10:12]

[[('An', 'False'),
  ('ethical', 'False'),
  ('sympathy', 'False'),
  ('in', 'False'),
  ('an', 'False'),
  ('artist', 'False'),
  ('is', 'False'),
  ('an', 'False'),
  ('unpardonable', 'False'),
  ('mannerism', 'False'),
  ('of', 'False'),
  ('style', 'True'),
  ('no', 'False'),
  ('artist', 'False'),
  ('is', 'False'),
  ('ever', 'False'),
  ('morbid', 'True')],
 [('The', 'False'),
  ('artist', 'False'),
  ('can', 'False'),
  ('express', 'False'),
  ('everything', 'True'),
  ('Thought', 'False'),
  ('and', 'False'),
  ('language', 'False'),
  ('are', 'False'),
  ('to', 'False'),
  ('the', 'False'),
  ('artist', 'False'),
  ('instruments', 'False'),
  ('of', 'False'),
  ('an', 'False'),
  ('art', 'True')]]

##### Створюю тренувальні токени та тренувальні лейбли

In [36]:
train_tokens = [[pair[0] for pair in sentence] for sentence in train_data]
train_labels = [[pair[1] for pair in sentence] for sentence in train_data]

with open('train_tokens.pickle', 'wb') as handle:
    pickle.dump(train_tokens, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('train_labels.pickle', 'wb') as handle:
    pickle.dump(train_labels, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Ngrams

##### Нграми збираю на Gutenberg корпусі (3035 файлів). Він більше за інші корпуси підходить під мої тренувальні дані (худ літ)

##### Збираю параграфи з усіх файлів директорії. Функція gather_paragraphs - розбиває пафаграфи по '\n\n',а потім додатково замінює перехід на нову лінію на пробіл у межах кожного параграфу. Це потрібно для подальшого коректного парсингу речень, так як текст у файлах має обмеження по кількості символів у однії лінії і має багато додаткових переходнів на нову лінію.

##### Функція gather_sentences - розбиває параграфи на речення 

##### Не додаю результат роботи коду до ноутбуку, так як він виконувався на іншому більш потужному ком'ютері (більше 3 годин)

In [ ]:
def gather_paragraphs(inp):
    paragraphs = []

    for i, filename in enumerate(os.listdir(inp)):
        with open(os.path.join(inp, filename), 'r', errors='ignore') as f: 
            paragraphs.extend(paragraph.replace('\n', ' ').strip() for paragraph in f.read().split('\n\n'))
    
    return paragraphs

def gather_sentences(paragraphs):
    result = []

    for doc in nlp.pipe(paragraphs, disable=["tagger", "ner", "textcat"]):
        for sent in doc.sents:
            text = sent.text
            if not re.search(r'["“]', text) and not re.match(r'[a-z,-]', text) \
            and not re.search(r'\b[A-Z]+\b', text):
                result.append(text)
    
    return result

In [ ]:
def make_ngrams(sentences, n):
    bigrams = defaultdict(int)
    lemma_bigrams = defaultdict(int)
    pos_bigrams = defaultdict(int)
    dep_bigrams = defaultdict(int)
    tag_bigrams = defaultdict(int)
    
    j = 0
    
    for sent in nlp.pipe(sentences, disable=["ner", "textcat"]):
        token_list = ['<S>']
        pos_list = ['<S>']
        dep_list = ['<S>']
        lemma_list = ['<S>']
        tag_list = ['<S>']
                
        for token in sent:
            if token.text.strip():
                token_list.append(token.text)
                pos_list.append(token.pos_)
                dep_list.append(token.dep_)
                lemma_list.append(token.lemma_)
                tag_list.append(token.tag_)

        for i in range(len(token_list) - n + 1):
            bigrams[tuple(token_list[i:i + n])] += 1
            pos_bigrams[tuple(pos_list[i:i + n])] += 1
            dep_bigrams[tuple(dep_list[i:i + n])] += 1
            lemma_bigrams[tuple(lemma_list[i:i + n])] += 1
            tag_bigrams[tuple(tag_list[i:i + n])] += 1

        if j % 20000 == 0 or j == len(sentences) - 1:
            print(datetime.datetime.now(), j, len(sentences) - j, 'left')
            with open('data/bigrams.pickle', 'wb') as handle:
                pickle.dump(bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)
            with open('data/lemma_bigrams.pickle', 'wb') as handle:
                pickle.dump(lemma_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)                
            with open('data/pos_bigrams.pickle', 'wb') as handle:
                pickle.dump(pos_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)
            with open('data/dep_bigrams.pickle', 'wb') as handle:
                pickle.dump(dep_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)
            with open('data/tag_bigrams.pickle', 'wb') as handle:
                pickle.dump(tag_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)                        

        j += 1
    
    return bigrams, pos_bigrams, dep_bigrams, lemma_bigrams, tag_bigrams